In [11]:
# Please execute/shift-return this cell everytime you run the notebook.  Don't edit it. 
%load_ext autoreload
%autoreload 2
from notebook import * 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Real case studies -- Intel Core i7 14th Generation

### Intel Core i7 14700K -- 12-way L1, 64B-blocked, 48KB cache

Let's run it without the above loop code to figure the baseline memory accesses without running the loop on a Jetson nano.

Let's again dump, parse and simulation the address sequence.

In [3]:
! cd 4way_madd; make madd_dump; cd ..; 
!echo "element,address"> addresses_madd.csv; 
!./4way_madd/madd_dump 8192 8192 2>> addresses_madd.csv
! head -n 101 addresses_madd.csv > addresses_digest.csv
df = pd.read_csv("addresses_digest.csv",skipfooter=1,engine='python')
df["address"] = df["address"].str.replace('0x','')
df["address"]=df[["address"]].apply(lambda x: x.astype(str).map(lambda x: int(x, base=16)))
# only show the first N addresses 
#N = 32
#df2 = df2.iloc[:N]
C = 49152
B = 64
A = 12
offset_bits = int(math.log2(B))
S = int(C/(B*A))
index_bits = int(math.log2(S))
df["tag"]=(df["address"].apply(lambda x: x >> (offset_bits+index_bits)))
df["tag"] = df["tag"].apply(lambda x: hex(x))
df["index"] = df["address"].apply(lambda x: hex((x>>offset_bits) % S))
df["address"] = df["address"].apply(lambda x: hex(x))
display_df_mono(df)

make: 'madd_dump' is up to date.
169317177,70735589,0.417770,0.237476,0.016798,0.000151,10904,72291569


,element,address,tag,index
0,a[0],0x79f8685af000,0x79f8685af,0x0
1,b[0],0x79f8685bf000,0x79f8685bf,0x0
2,c[0],0x79f8685cf000,0x79f8685cf,0x0
3,d[0],0x79f8685df000,0x79f8685df,0x0
4,e[0],0x79f8685ef000,0x79f8685ef,0x0
5,a[1],0x79f8685af008,0x79f8685af,0x0
6,b[1],0x79f8685bf008,0x79f8685bf,0x0
7,c[1],0x79f8685cf008,0x79f8685cf,0x0
8,d[1],0x79f8685df008,0x79f8685df,0x0
9,e[1],0x79f8685ef008,0x79f8685ef,0x0


#### Run with performance counters

In [8]:
# Run it "without" the above code.
! make -C ./4way_madd/ madd_intel 
! echo -n "Intel Core i7 13700," >> ./4way_madd/4way_add.csv
! lscpu; cd ~/courses/CSE142/demo/memory/4way_madd/; ./madd_intel 16384 16384 >> 4way_add.csv

make: Entering directory '/nfshome/htseng/courses/CSE142/demo/memory/4way_madd'
make: 'madd_intel' is up to date.
make: Leaving directory '/nfshome/htseng/courses/CSE142/demo/memory/4way_madd'
Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          39 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   24
  On-line CPU(s) list:    0-23
Vendor ID:                GenuineIntel
  Model name:             13th Gen Intel(R) Core(TM) i7-13700F
    CPU family:           6
    Model:                183
    Thread(s) per core:   2
    Core(s) per socket:   16
    Socket(s):            1
    Stepping:             1
    CPU(s) scaling MHz:   24%
    CPU max MHz:          5200.0000
    CPU min MHz:          800.0000
    BogoMIPS:             4224.00
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 s
   

Let's run it with the above loop code again and observe the changes in L1 cache misses/accesses

#### Run with the 4-way matrix add loop code.

In [13]:
# Run it "with" the above code.
! cd ~/courses/CSE142/demo/memory/4way_madd/; make madd_intel ; valgrind --tool=cachegrind ./madd_intel 16384 16384  >& intel_with_loop.perf; grep 'D   refs\|D1' intel_with_loop.perf

make: 'madd_intel' is up to date.


In [11]:
# Let's do some math here
total_number_of_accesses_before_the_loop =2285911
total_number_of_accesses_after_the_loop = 2326877
total_number_of_accesses_in_the_loop = total_number_of_accesses_after_the_loop-total_number_of_accesses_before_the_loop
total_number_of_misses_before_the_loop = 13447
total_number_of_misses_after_the_loop = 23694
total_number_of_misses_in_the_loop = total_number_of_misses_after_the_loop-total_number_of_misses_before_the_loop
miss_rate_of_the_loop = total_number_of_misses_in_the_loop/total_number_of_accesses_in_the_loop

print(f"access in the loop: %d misses in the loop %d miss_rate %lf" % (total_number_of_accesses_in_the_loop, total_number_of_misses_in_the_loop, miss_rate_of_the_loop))

access in the loop: 40966 misses in the loop 10247 miss_rate 0.250134


In [14]:
display_df_mono(render_csv("./4way_madd/4way_add.csv"))

,index,CPU,IC,Cycles,CPI,CT,ET,miss_rate,misses,accesses
0,0,Jetson Nano,0,0,nan,inf,0.000308,nan,0,0
1,1,Intel Core i7 13700,0,0,nan,inf,0.000032,nan,0,0
2,2,Intel Core i7 13700,0,0,nan,inf,0.000032,nan,0,0
3,3,Intel Core i7 13700,0,0,nan,inf,0.000032,nan,0,0


In [18]:
! ./4way_madd/madd_intel 131072 131072
! cat ./4way_madd/4way_add.csv

737647,486419,0.659420,0.180914,0.000088,0.120604,40320,334318
CPU, IC, Cycles, CPI, CT, ET, miss_rate, misses, accesses
Jetson Nano,0,0,nan,inf,0.000308,nan,0,0
Intel Core i7 13700,0,0,-nan,inf,0.000032,-nan,0,0
Intel Core i7 13700,0,0,-nan,inf,0.000032,-nan,0,0
Intel Core i7 13700,0,0,-nan,inf,0.000032,-nan,0,0
